<a href="https://colab.research.google.com/github/LeeSeungwon89/Machine-learning_Theory/blob/master/5-3%20%ED%8A%B8%EB%A6%AC%EC%9D%98%20%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **트리의 앙상블**

## **정형 데이터와 비정형 데이터**

**정형 데이터(structured data)**는 구조화된 데이터입니다. CSV, Database, Excel에 저장하기 쉬운 형태입니다.

**비정형 데이터(unstructured data)**는 비구조화된 데이터입니다. 텍스트 데이터, 사진, 디지털 음악 등을 의미합니다. 규칙성을 찾기 어려워 전통적인 머신러닝 방법으로는 모델을 만들기 어렵습니다. 신경망 알고리즘을 활용하여 모델을 만들어야 합니다.

## **앙상블 학습**

**앙상블 학습(Ensemble learning)**의 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 결정 트리 기반 머신러닝 알고리즘입니다. **정형 데이터**를 다루는 데 가장 뛰어난 성과를 냅니다. 참고로 'Ensemble'은 '전체적인 어울림이나 통일', '합주단' 등을 의미합니다.

## **데이터 준비**

모델을 만들 데이터를 준비하겠습니다. 와인 데이터셋입니다.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

## **랜덤 포레스트**

**랜덤 포레스트(Random Forest)**는 앙상블 학습을 적용할 때 가장 먼저 시도하길 권장하는 모델이며 매우 안정적인 성능을 냅니다. 결정 트리 여러 개를 랜덤하게 만들어 숲을 만들고, 각 결정 트리의 예측을 사용하여 최종 예측을 만드는 방식을 취합니다. 

각 트리를 훈련하기 위한 데이터는 입력한 훈련 데이터에서 샘플을 랜덤하게 추출하여 만듭니다. 이때 한 샘플이 중복되어 추출될 가능성도 존재합니다. 예컨대 가방 1,000개에서 샘플을 100개씩 뽑는다면 먼저 1개를 뽑고, 뽑았던 1개를 다시 가방에 넣습니다. 계속 100개를 뽑다 보면 중복된 샘플을 뽑을 수도 있습니다. 

이렇게 중복되게 뽑은 샘플을 **부트스트랩 샘플(Bootstrap sample)**이라고 부릅니다. **부트스트랩(Bootstrap)**은 보통 **부트스트랩 방식**이라고 부르며, 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식입니다. 기본적으로 부트스트랩 샘플은 훈련 세트 크기와 같게 만듭니다. 가방 1,000개에서 샘플 1,000개를 중복하면서 뽑기 때문에 부트스트랩 샘플은 훈련 세트와 크기가 같습니다. 

아울러 랜덤 포레스트는 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾습니다.

랜덤 포레스트를 구현한 분류 클래스는 `RandomForestClassifier`입니다. 전체 특성 개수의 제곱근만큼 특성을 사용합니다. 다시 말하면 특성이 4개 있다면 노드마다 2개를 랜덤하게 선택하여 사용합니다. 결정 트리의 앙상블이므로 결정 트리를 구현한 클래스인 `DecisionTreeClassifier`가 가진 주요 매개변수인 `criterion`, `max_depth`, `max_feature`, `min_samples_split`, `min_impurity_decrease`, `min_samples_leaf` 등을 모두 제공합니다. 아래는 `RandomForestClassifier` 클래스가 가진 매개변수 목록입니다.

- `n_estimators`: 앙상블을 구성할 트리 개수를 지정하는 매개변수입니다. 기본값은 `100`입니다.

- `criterion`: 불순도를 지정하는 매개변수입니다. 기본값은 지니 불순도를 의미하는 `gini`이며, 엔트로피 불순도를 사용하려면 `entrophy`로 지정합니다.

- `max_depth`: 트리가 성장할 최대 깊이를 지정하는 매개변수입니다. 기본값은 `None`으로 리프 노드가 순수하거나 `min_samples_split` 매개변수에 지정한 값보다 샘플 개수가 적을 때까지 성장합니다.

- `min_samples_split`: 노드를 나누기 위한 최소 샘플 개수를 지정하는 매개변수입니다. 기본값은 `2`입니다.

- `max_features`: 최적의 분할을 위해 탐색할 특성 개수를 지정하는 매개변수입니다. 기본값은 `auto`로 특성 개수의 제곱근을 의미합니다.

- `bootstrap`: 부트스트랩 샘플 사용 여부를 지정하는 매개변수입니다. 기본값은 `True`입니다.

- `oob_score`: OOB 샘플을 사용하여 훈련한 모델을 평가할지 여부를 지정하는 매개변수입니다. 기본값은 `False`입니다. 아래에서 추가로 설명하겠습니다.

- `n_jobs`: 병렬 실행에 사용할 CPU 코어 수를 지정하는 매개변수입니다. 기본값은 `1`로 하나의 코어를 사용하며, `-1`로 지정하면 시스템에 있는 모든 코어를 사용합니다.

랜덤 포레스트는 기본적으로 결정 트리 100개를 훈련합니다. 분류일 경우에는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼고, 회귀일 경우에는 각 트리의 예측을 평균한 값을 예측으로 삼습니다. 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 방지하고, 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있습니다. 종종 기본 매개변수 설정만으로도 좋은 결과를 낼 수 있습니다.

결정 트리의 큰 장점 중 하나인 특성 중요도를 계산하기도 합니다. 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것입니다.

참고로 `RandomForestRegressor`는 회귀를 다루는 랜덤 포레스트 클래스입니다. `RandomForestClassifier` 클래스가 전체 특성 개수의 제곱근만큼 특성을 사용한다면, `RandomForestRegressor` 클래스는 전체 특성을 사용합니다.

### **교차 검증 수행하기**

교차 검증을 수행하여 최상의 검증 점수를 확인하겠습니다. 매개변수는 `n_jobs`만 `-1`로 바꾸겠습니다.

In [4]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)

scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.9973541965122431
검증 세트 점수: 0.8905151032797809


과대적합 상태입니다. 알고리즘을 살펴보는 것이 목적이므로 매개변수를 더 조정하지는 않겠습니다. 이 예제는 매우 간단하고 특성이 많지 않아서 그리드 서치를 사용하더라도 하이퍼파라미터 튜닝 결과는 크게 나아지지 않습니다.

### **모델 훈련 후 특성 중요도 출력하기**

모델을 훈련한 후 특성 중요도를 출력하겠습니다. 특성 중요도는 `feature_importances_` 속성에 저장되어 있습니다.

In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


출력된 특성 중요도는 각각 'alcohol', 'sugar', 'pH'를 나타냅니다. 앞의 1절에서 결정 트리 모델의 특성 중요도인 `[0.12345626, 0.86862934, 0.0079144]`와 랜덤 포레스트 점수를 비교하면 'alcohol'과 'pH'의 중요도가 상승하고 'sugar'의 중요도가 감소했습니다. 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련했기 때문입니다. 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 부여한 것입니다. 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 됩니다.

### **OOB 샘플로 모델 평가하기**

`RandomForestClassifier` 클래스는 자체적으로 모델을 평가하는 점수를 얻을 수 있습니다. 부트스트랩 샘플에 포함되지 않고 남는 샘플을 **OOB(out of bag)**이라고 부르는데, 부트스트랩 샘플로 훈련한 결정 트리를 이 OOB를 사용하여 평가합니다. 검증 세트 역할을 한다고 볼 수 있습니다. 교차 검증을 대신하므로 훈련 세트에 더 많은 샘플을 사용할 수 있습니다. `RandomForestClassifier` 클래스의 `oob_score` 매개변수를 `True`로 지정하면 OOB 점수를 평균하여 이 점수를 얻을 수 있습니다.

In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


교차 검증에서 얻은 점수인 `0.8905151032797809`와 비슷한 결과를 얻었습니다.

## **엑스트라 트리**

- '엑스트라 트리(Extra Trees)'는 랜덤 포레스트와 매우 비슷하게 작동하는 모델임. 기본적으로 100개의 결정 트리를 훈련하며, 랜덤 포레스트와 동일하게 결정 트리가 제공하는 매개변수 대부분을 지원함. 

- 일부 특성을 랜덤하게 선택하여 노드를 분할하는 데 사용함.

- 결정 트리를 만들 때 전체 훈련 세트를 사용하므로, 부트스트랩 샘플을 사용하지 않음.

- 노드를 분할할 때 가장 좋은 분할을 찾지 않고 무작위로 분할함. 이렇게 무작위로 분할하면 모델 성능이 낮아지겠지만, 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트 점수를 높이는 효과가 있음. 아래에서 추가로 설명함.

- `DecisionTreeClassifier` 클래스의 `splitter` 매개변수를 `random`으로 지정하면 엑스트라 트리와 동일함.

- `ExtraTreesClassifier`: 분류 클래스.

   - `n_estimators`: 앙상블을 구성할 트리 개수를 지정하는 매개변수. 기본값은 `100`.

   - `criterion`: 불순도를 지정하는 매개변수. 기본값은 지니 불순도를 의미하는 `gini`이며, 엔트로피 불순도를 사용하려면 `entrophy`로 지정함.

   - `max_depth`: 트리가 성장할 최대 깊이를 지정하는 매개변수. 기본값은 `None`으로 리프 노드가 순수하거나 `min_samples_split` 매개변수에 지정한 값보다 샘플 개수가 적을 때까지 성장함.

   - `min_samples_split`: 노드를 나누기 위한 최소 샘플 개수를 지정하는 매개변수. 기본값은 `2`.

   - `max_features`: 최적의 분할을 위해 탐색할 특성 개수를 지정하는 매개변수. 기본값은 `auto`로 특성 개수의 제곱근을 의미함.

   - `bootstrap`: 부트스트랩 샘플 사용 여부를 지정하는 매개변수. 기본값은 `False`로 랜덤 포레스트와 다름.

   - `oob_score`: OOB 샘플을 사용하여 훈련한 모델을 평가할지 여부를 지정하는 매개변수. 기본값은 `False`.

   - `n_jobs`: 병렬 실행에 사용할 CPU 코어 수를 지정하는 매개변수. 기본값은 `1`로 하나의 코어를 사용하며, `-1`로 지정하면 시스템에 있는 모든 코어를 사용함.

- `ExtraTreesRegressor`: 회귀 클래스.

### **교차 검증 수행하기**

In [ ]:
# 교차 검증을 수행함.
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 랜덤 포레스트와 비슷한 결과를 출력함. 이 예제는 특성이 많지 않아서 두 모델의 차이가 크지 않음.
# 보통 엑스트라 트리가 무작위성이 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 함.
# 더 많은 결정 트리를 훈련하지만 노드를 랜덤하게 분할하므로 계산 속도가 빠름.

0.9974503966084433 0.8887848893166506


### **모델 훈련 후 특성 중요도 출력하기**

In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)
# 앞의 1절에서 결정 트리 모델의 특성 중요도는 [0.12345626, 0.86862934, 0.0079144] 이고, 각각 alcohol, sugar, pH를 나타냄.
# 엑스트라 트리의 점수와 비교하면 alcohol과 pH의 중요도가 상승하고 sugar의 중요도가 감소함. 

[0.20183568 0.52242907 0.27573525]


## **그레이디언트 부스팅**

- '그레이디언트 부스팅(Gradient boosting)'은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법이며, 그레이디언트(Gradient)의 의미처럼 '경사 하강법'을 사용하여 트리를 앙상블에 추가함. 이전에 설명했듯이 '경사 하강법'은 손실 함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정임.

- 모델의 가중치와 절편을 조금씩 바꾸면서 가장 낮은 곳으로 내려옴. 손실 함수의 낮은 곳으로 천천히 조금씩 이동하기 위해 깊이가 얕은 결정 트리를 계속 추가하면서 이동하는 방법을 취함.

- 분류에서는 '로지스틱 손실 함수'를 사용하고, 회귀에서는 '평균 제곱 오차 함수'를 사용함.

- `GradientBoostingClassifier`: 분류 클래스이며, 기본적으로 깊이가 3인 결정 트리를 100개 사용하는  깊이가 얕은 결정 트리를 사용하므로 과대적합에 강하고 높은 일반화 성능을 낼 수 있음.

 - `loss`: 손실 함수를 지정하는 매개변수. 기본값은 `deviance`로 로지스틱 손실 함수를 의미함.

 - `n_estimators`: 부스팅 단계를 수행하는 트리 개수를 지정하는 매개변수. 기본값은 `100`.

 - `learning_rate`: 트리가 앙상블에 기여하는 정도인 학습률을 지정하는 매개변수. 속도를 조절할 수 있으며 기본값은 `0.1`.

 - `subsample`: 트리 훈련에 사용하는 훈련 세트의 비율을 정하는 매개변수. 기본값은 `1.0`으로 전체 훈련 세트를 사용함. 1.0보다 작으면 훈련 세트 일부를 사용하며, 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택하여 진행하는 '확률적 경사 하강법'이나 '미니배치 경사 하강법'과 비슷함.

 - `max_depth`: 개별 회귀 트리의 최대 깊이를 지정하는 매개변수. 기본값은 `3`.

- `GradientBoostingRegressor`: 회귀 클래스.

- 일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 내지만, 순서대로 트리를 추가하기 때문에 훈련 속도가 느림. 즉 `GradientBoostingClassifier` 클래스에는 `n_jobs` 매개변수가 없음.

### **교차 검증 수행하기**

In [ ]:
# `GradientBoostingClassifier` 클래스를 사용하여 교차 검증 점수를 확인함.
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대적합이 거의 되지 않음.
# 그레이디언트 부스팅은 결정 트리 개수를 늘려도 과대적합에 매우 강함.
# 학습률을 증가시키고 트리 개수를 늘리면 성능이 더 향상될 수 있음.

0.8881086892152563 0.8720430147331015


### **매개변수를 조정하여 점수 출력하기**

In [ ]:
# `n_estimators` 매개변수 값을 변경하여 결정 트리 개수를 500개로 늘리고
# `learning_rate` 매개변수 값을 변경하여 학습률을 0.2로 지정함.
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


### **모델 훈련 후 특성 중요도 출력하기**

In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
# 그레이디언트 부스팅은 sugar에 0.68010884만큼 집중하며,
# 랜덤 포레스트의 sugar에 대한 집중도(0.50039841)보다 더 비중을 두고 집중함.

[0.15872278 0.68010884 0.16116839]


## **히스토그램 기반 그레이디언트 부스팅**

- '히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)'은 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘임.

- 입력 특성을 256개 구간으로 나눠서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾음. 256개 구간 중에 하나를 떼어 놓고 누락된 값을 위해 사용하며, 입력에 누락된 특성이 있더라도 따로 전처리할 필요가 없음.

- `HistGradientBoostingClassifier`: 분류 클래스이며, 일반적으로 기본 매개변수로 안정적인 성능을 얻을 수 있음.

 - `learning_rate`: 학습률(감쇠율)을 지정하는 매개변수. 기본값은 `0.1`이며, 1.0이면 감쇠가 전혀 없음. 여기서 '감쇠'는 '힘이 줄어서 약하여짐'을 의미함.

 - `max_iter`: 부스팅 단계를 수행하는 트리 개수(부스팅 반복 횟수)를 지정하는 매개변수. 기본값은 `100`. 모델 성능을 높이려면 이 매개변수를 테스트 함. 트리 개수를 지정하는 `n_estimators` 매개변수 대신 사용함. 

 - `max_bins`: 입력 데이터를 나눌 구간 개수를 지정하는 매개변수. 기본값은 255이며 더 크게 지정할 수 없음. 위에서 설명한 256개 구간처럼, 누락된 값을 위해 구간 1개가 추가됨.

- `HistGradientBoostingRegressor`: 회귀 클래스.

### **교차 검증 수행하기**

In [ ]:
# 히스토그램 기반 그레이디언트 부스팅은 아직 테스트 과정에 속해 있으며,
# 사용하려면 `sklearn.experimental` 패키지에 있는 `enable_hist_gradient_boosting` 모듈을 임포트 해야 함.
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대적합을 제어하면서 그레이디언트 부스팅보다 좀 더 높은 성능을 냄.

0.9321723946453317 0.8801241948619236


### **모델 훈련 후 특성 중요도 출력하기**

- 특성 중요도를 계산하기 위해 `permutation_importance()` 함수를 사용함. `permutation`은 '순열, 치환'을 의미함.

- 특성을 하나씩 랜덤하게 섞어서 모델 성능이 변화하는지 관찰하여 어떤 특성이 중요한지 계산함.

 - `n_repeats`: 랜덤하게 섞을 횟수를 지정하는 매개변수.

- 객체 3가지를 가짐.

 - `improtances`: 반복하여 얻은 특성 중요도.

 - `importances_mean`: 반복하여 얻은 특성 중요도의 평균.

 - `importances_std`: 반복하여 얻은 특성 중요도의 표준편차.

- 훈련 세트뿐만 아니라 테스트 세트에서 적용할 수 있고, 사이킷런에서 제공하는 '추정기' 모델에도 적용 가능함.

In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances)
print(result.importances_mean)
print(result.importances_std)
# 이중에 평균은 랜덤 포레스트와 비슷한 비율을 나타냄.

[[0.08793535 0.08350972 0.08908986 0.08312488 0.09274581 0.08755051
  0.08601116 0.09601693 0.09082163 0.09082163]
 [0.22782374 0.23590533 0.23936887 0.23436598 0.23725226 0.23436598
  0.23359631 0.23398114 0.23994612 0.22724649]
 [0.08581874 0.08601116 0.08062344 0.07504329 0.08427939 0.07792957
  0.07234943 0.07465846 0.08139311 0.08466423]]
[0.08876275 0.23438522 0.08027708]
[0.00382333 0.00401363 0.00477012]


In [ ]:
# 테스트 세트로 특성 중요도를 계산함.
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances)
print(result.importances_mean)
print(result.importances_std)
# 그레이디언트 부스팅과 비슷하게 좀 더 sugar에 집중함.
# 이런 분석 과정을 통해 실전에 투입했을 때 어떤 특성에 관심을 가질지 예상할 수 있음.

[[0.06230769 0.05769231 0.05538462 0.05538462 0.06076923 0.06076923
  0.06846154 0.06230769 0.05461538 0.05923077]
 [0.20076923 0.2        0.21153846 0.20076923 0.20307692 0.18923077
  0.19615385 0.19461538 0.21384615 0.21384615]
 [0.05692308 0.04692308 0.05076923 0.04769231 0.04692308 0.05
  0.04384615 0.04692308 0.04307692 0.05692308]]
[0.05969231 0.20238462 0.049     ]
[0.004      0.007938   0.00453846]


### **테스트 세트로 성능 확인하기**

In [ ]:
hgb.score(test_input, test_target)
# 실전에 투입하면 이보다 더 낮은 성능을 가질 가능성이 높음.
# 결론적으로 앙상블 모델은 단일 결정 트리 모델보다 좋은 결과를 얻음.
# 2절 랜덤 서치에서 테스트 정확도는 86%였음.

0.8723076923076923

### **XGBoost**

- 히스토그램 기반 그레이디언트 부스팅을 구현한 대표적인 라이브러리.

- `tree_method` 매개변수를 `hist`로 지정하여 히스토그램 기반 그레이디언트 부스팅을 사용함.

- `cross_validate()` 함수와 함께 사용할 수 있음.

- 더 자세한 설명은 [[1]](https://xgboost.ai/), [[2]](https://xgboost.readthedocs.io/en/latest/)를 참고.

#### **교차 검증 수행하기**

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


#### **모델 훈련 후 특성 중요도 출력하기**

In [ ]:
from sklearn.inspection import permutation_importance

xgb.fit(train_input, train_target)
result = permutation_importance(xgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances)
print(result.importances_mean)
print(result.importances_std)

[[0.03983067 0.03521262 0.04329421 0.039061   0.0421397  0.03925342
  0.03713681 0.04348663 0.04252453 0.04310179]
 [0.19607466 0.20069271 0.19973061 0.20030787 0.20319415 0.20319415
  0.19665191 0.19395805 0.20261689 0.19415047]
 [0.03675197 0.03636713 0.0359823  0.03232634 0.03790648 0.03617472
  0.03386569 0.02944006 0.03867616 0.03790648]]
[0.04050414 0.19905715 0.03553973]
[0.00271338 0.00341103 0.00272264]


#### **테스트 세트로 성능 확인하기**

In [ ]:
xgb.score(test_input, test_target)

0.8669230769230769

### **LightBGM**

- 히스토그램 기반 그레이디언트 부스팅을 구현한 대표적인 라이브러리.

- `tree_method` 매개변수를 `hist` 로 지정하여 히스토그램 기반 그레이디언트 부스팅을 사용함.

- `cross_validate()` 함수와 함께 사용할 수 있음.

- 더 자세한 설명은 [[1]](https://github.com/microsoft/LightGBM), [[2]](https://lightgbm.readthedocs.io/en/latest)를 참고.

#### **교차 검증 수행하기**

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(tree_method='hist', random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293


#### **모델 훈련 후 특성 중요도 출력하기**

In [ ]:
from sklearn.inspection import permutation_importance

lgb.fit(train_input, train_target)
result = permutation_importance(lgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances)
print(result.importances_mean)
print(result.importances_std)

[[0.08908986 0.08389455 0.09024437 0.08524149 0.09582451 0.09101405
  0.09005195 0.09390033 0.09274581 0.09332307]
 [0.23051761 0.23513566 0.24110063 0.23224937 0.2376371  0.23609775
  0.23571291 0.2345584  0.24013854 0.22705407]
 [0.08716567 0.08697325 0.08774293 0.0802386  0.0894747  0.07869925
  0.07562055 0.07869925 0.08735809 0.08601116]]
[0.090533   0.2350202  0.08379835]
[0.00355975 0.00404098 0.00467245]


#### **테스트 세트로 성능 확인하기**

In [ ]:
lgb.score(test_input, test_target)

0.8638461538461538